# Libraries

In [ ]:
import os
import rioxarray as rxr
from shapely.geometry import mapping
import geopandas as gpd
import rasterio as rio
from rioxarray.merge import merge_arrays

In [2]:
# Set the base directory with an environment variable, fallback to a relative path
main_dir = os.getenv('PROJECT_DIR', os.path.join(os.getcwd(), 'data', 'ANIN', 'Generating Indices', 'VCI', 'NDVI-16', 'Trial', 'Tiles'))
#Tiles_path = os.path.join(main_dir, 'Tiles')
Tile_1 = os.path.join(main_dir, 'tile_1')
Tile_2 = os.path.join(main_dir, 'tile_2')
Tile_3 = os.path.join(main_dir, 'tile_3')
Tile_4 = os.path.join(main_dir, 'tile_4')

Images_in_tile1 = sorted(os.listdir(Tile_1))
Images_in_tile2 = sorted(os.listdir(Tile_2))
Images_in_tile3 = sorted(os.listdir(Tile_3))
Images_in_tile4 = sorted(os.listdir(Tile_4))

In [ ]:
Boundary = os.path.join(main_dir, 'Boundary')
South_Africa = gpd.read_file(os.path.join(Boundary,'ZAF_adm0.shp'))
lesotho = gpd.read_file(os.path.join(Boundary, 'LSO_adm0.shp'))
eswatini = gpd.read_file(os.path.join(Boundary, 'swz_admbnda_adm0_CSO_2007.shp'))
merged = South_Africa.geometry.append(eswatini.geometry)
merged =merged.geometry.append(lesotho.geometry)

In [5]:
Clipped_path = os.path.join(main_dir, 'Clipped_trial')
for i in range (0 , len(Images_in_tile4)):
    tile1 = rxr.open_rasterio(os.path.join(Tile_1, Images_in_tile1[i], masked=True))
    tile2 = rxr.open_rasterio(os.path.join(Tile_2, Images_in_tile2[i], masked=True))
    tile3 = rxr.open_rasterio(os.path.join(Tile_3, Images_in_tile3[i], masked=True))
    tile4 = rxr.open_rasterio(os.path.join(Tile_4, Images_in_tile4[i], masked=True))
    merged_raster = merge_arrays(dataarrays = [tile1['250m 16 days NDVI'], tile2['250m 16 days NDVI'],
                                               tile3['250m 16 days NDVI'],tile4['250m 16 days NDVI']])
    clipped = merged_raster.rio.clip(merged.geometry.apply(mapping),
                                     crs=merged.crs,
                                     all_touched=True,
                                     from_disk=True).squeeze()
    year = os.path.join(Clipped_path, Images_in_tile1[i][9:13])
    check_folder_year = os.path.isdir(year)
    if not check_folder_year:
        os.makedirs(year)
    
    clipped.rio.to_raster(os.path.join(year, Images_in_tile1[i]+ '.tif'))